## Pandas analysis

In the following a series of exercises is proposed on a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGA's. Each measurement (i.e. each raw) consists of the address of the TDC providing the signal, 'FPGA' and 'TDC_Channel, and the timing information itself, 'ORBIT_CNT', 'BX_COUNTER' and 'TDC_MEAS'. Each TDC count correspond 25/30 ns, whereas the BX_COUNTER feauters gets updated every 25 ns and the ORBIT_CNT every 'x' BX_COUNTER. You can see these way of storing the time as similar to hours, minutes and seconds.

1\. Create a Pandas DataFrame by read N raws of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of raws and larger that 10k.

2\. Find out the value of 'x'

3\. Find out how much the data taking lasted. You can either make an estimate on the baseis of the fraction of the measurements (raws) you read, or perform this check precisely by reading out the whole dataset

4\. Create a new column with the actual time in ns (as a combination of the other three columns with timing information)

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1

6\. Create a new DataFrame with only the raws with HEAD=1

7\. Make two occupancy plots (one per FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as npr
#npr.seed(63)

In [ ]:
#1

#file_name = "/home/spiro/home/eminentc/Documents/LaboratoryOfcomputationalPhysics/data/data_000637.txt"
file_name = "/Users/clara/Documents/LaboratoryOfComputationalPhysics/data/data_000637.txt"
data=pd.read_csv(file_name)
n_max = data.shape[0]
n_min = 10000

In [ ]:
N = npr.randint(n_min,n_max)
print(N)
data = data.iloc[:N,]
data


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(data.loc[4:1000,'BX_COUNTER'])
#plt.plot(data.loc[4:1000,'ORBIT_CNT'])

In [ ]:
#2
x = data.BX_COUNTER.max()
print("Period:",x)

In [ ]:
#3
partial_time = (data.ORBIT_CNT.max() - data.ORBIT_CNT.min())*25*x + data.BX_COUNTER.iloc[-1]*25 + data.TDC_MEAS.iloc[-1]*(25/30)
print("Partial time: %f ns" %partial_time)
print("Total time estimated: %f s" %(partial_time*n_max/(N*10**9)))

In [ ]:
#4
t_in = data.ORBIT_CNT.min()
data['TIME'] = (data.ORBIT_CNT - t_in)*25*x + data.BX_COUNTER*25 + data.TDC_MEAS*(25/30)
#data


In [ ]:
%matplotlib inline 

#plt.plot(data.loc[4:1000,['BX_COUNTER','ORBIT_CNT']])
plt.plot(data.loc[4:1000,'TIME'])


In [ ]:
#5
import numpy.random as npr
npr.seed(63)
data.loc[:,'HEAD'] = npr.randint(0,2,size=len(data))
#data


In [ ]:
#6
data2=data.copy()
data2=data2[data2.HEAD == 1]
#data2

In [ ]:
#7 & 8
#FPGA1
fpga1 = data[data.FPGA == 1].loc[:,['TDC_CHANNEL']]

fpga1['COUNTER'] = np.ones(len(fpga1))
fpga1_in_order = fpga1.groupby('TDC_CHANNEL').sum()

plt.plot(fpga1_in_order.loc[:,'COUNTER'])
plt.xlabel('TDC_CHANNEL')
plt.ylabel('COUNTS')

noisy1 = fpga1_in_order.sort_values('COUNTER')
print(noisy1.tail(3))

In [ ]:
#7 & 8
#FPGA 0
fpga0 = data[data.FPGA == 0].loc[:,['TDC_CHANNEL']]

fpga0['COUNTER'] = np.ones(len(fpga0))
fpga0_in_order = fpga0.groupby('TDC_CHANNEL').sum()

plt.plot(fpga0_in_order.loc[:,'COUNTER'])
plt.xlabel('TDC_CHANNEL')
plt.ylabel('COUNTS')

noisy0 = fpga0_in_order.sort_values('COUNTER')
print(noisy0.tail(3))


In [ ]:
#9
num_orbits = data.ORBIT_CNT.max() - data.ORBIT_CNT.min()
print('Number of unique orbits:', num_orbits)

In [ ]:
orbits = data[data.TDC_CHANNEL == 139].loc[:,['ORBIT_CNT']]
len(orbits.groupby("ORBIT_CNT"))
